In [1]:
!pip list | grep lang

google-ai-generativelanguage             0.6.6
langchain                                0.2.17
langchain-aws                            0.1.18
langchain-cohere                         0.1.9
langchain-community                      0.2.19
langchain-core                           0.2.43
langchain-experimental                   0.0.65
langchain-google-genai                   1.0.10
langchain-openai                         0.1.25
langchain-text-splitters                 0.2.4
langdetect                               1.0.9
langgraph                                0.2.46
langgraph-checkpoint                     2.0.3
langgraph-sdk                            0.1.35
langsmith                                0.1.142


In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [2]:
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain.agents import tool
from typing import TypedDict, Annotated
import operator

### Defining Tools

In [3]:
@tool
def calculate_si(p, r, t):
    """This function calculates the simple interest given the principal amount, rate of interest and time period."""
    return (p * r * t) / 100

@tool
def calculate_ci(p, r, t):
    """This function calculates the compound interest given the principal amount, rate of interest and time period."""
    return p * (1 + r / 100) ** t - p

In [4]:
print(type(calculate_si))

<class 'langchain_core.tools.structured.StructuredTool'>


### Defining Agent State

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

### Defining Agennt

In [6]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        ## creating graph
        graph = StateGraph(AgentState)
        ## adding nodes
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        ## adding conditional edges
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        ## adding edges
        graph.add_edge("action", "llm")
        ## setting entry point
        graph.set_entry_point("llm")
        ## compiling graph
        self.graph = graph.compile()
        ## binding tools to the model
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        print(f"Checking for action: {result}")
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
from models.llm import LLM

model = LLM('gpt-4')
llm = model.load_model()

In [8]:
prompt = """You are a smart accountant. Use the calculate simple interest and compound interest tools to calculate SI and CI. \
You are allowed to make multiple calls (either together or in sequence). \
"""

cbot = Agent(llm, [calculate_si, calculate_ci], system=prompt)

In [9]:
messages = [HumanMessage(content="What is the simple interest on a loan of 1000 USD at 5% for 10 years?")]
result = cbot.graph.invoke({"messages": messages})

Checking for action: content='' additional_kwargs={'tool_calls': [{'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'function': {'arguments': '{\n"p": 1000,\n"r": 5,\n"t": 10\n}', 'name': 'calculate_si'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 165, 'total_tokens': 192, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='run-8789db10-fb7a-42fa-a997-cfb02d5880db-0' tool_calls=[{'name': 'calculate_si', 'args': {'p': 1000, 'r': 5, 't': 10}, 'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 165, 'output_tokens': 27, 'total_tokens': 192, 'input_token_details': {}, 'output_token_details': {}}
Calling: {'name': 'calculate_si', 'args': {'p': 1000, 'r': 5, 't': 10}, 'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'type': 'tool_call'}
Back to the 

In [10]:
messages = [HumanMessage(content="What is the difference between simple interest and compound interest on a loan of 1000 USD at 5% for 10 years?")]
result = cbot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Checking for action: content='' additional_kwargs={'tool_calls': [{'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'function': {'arguments': '{\n"p": 1000,\n"r": 5,\n"t": 10\n}', 'name': 'calculate_si'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 170, 'total_tokens': 197, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='run-94477eb8-1f82-495b-afb4-d0c35f8a29a8-0' tool_calls=[{'name': 'calculate_si', 'args': {'p': 1000, 'r': 5, 't': 10}, 'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 170, 'output_tokens': 27, 'total_tokens': 197, 'input_token_details': {}, 'output_token_details': {}}
Calling: {'name': 'calculate_si', 'args': {'p': 1000, 'r': 5, 't': 10}, 'id': 'call_I2jChIUrjWAQeyv3s6wGk9nb', 'type': 'tool_call'}
Back to the 